In [1]:
import glob
import os
import pandas as pd
import numpy as np
import weaviate
import chromadb

In [2]:
chroma_client = chromadb.Client()

In [ ]:
weaviate_client = weaviate.Client("http://localhost:8080/")
weaviate_client.is_ready()

In [4]:
def load_parquet(parquet_path):
    return pd.read_parquet(parquet_path)

In [5]:
def print_parquet_data(data):
    for _, row in data.iterrows():
        print("Abstract:", row['abstract'])
        print("Embedding:", row['embeddings'])
        print("DOI:", row['doi'])
        print()

In [6]:
abstracts1 = load_parquet('arxiv-paper-abstracts/arxiv_abstracts/arxiv_abstracts/arxiv_abstracts/abstracts_1.parquet')

In [10]:
len(abstracts1['abstract'])

100000

In [11]:
collection1 = chroma_client.create_collection(name="abstracts1")

In [14]:
test_batch = abstracts1[:4]

,abstract,embeddings,doi
0,A fully differential calculation in perturba...,"[-0.035151865, 0.022851437, 0.025942933, -0.02...",0704.0001
1,"We describe a new algorithm, the $(k,\ell)$-...","[0.035485767, -0.0015772493, -0.0016615744, -0...",0704.0002
2,The evolution of Earth-Moon system is descri...,"[-0.014510429, 0.010210799, 0.049661566, -0.01...",0704.0003
3,We show that a determinant of Stirling cycle...,"[0.029191103, 0.047992915, -0.0061754594, -0.0...",0704.0004


In [17]:
for _, row in test_batch.iterrows():
    collection1.add(
        embeddings = list(row['embeddings']),
        documents = row['abstract'],
        ids = row['doi']
    )

ValueError: Expected embeddings to be a list, got [-3.51518653e-02  2.28514373e-02  2.59429328e-02 -2.09794827e-02
 -4.78580333e-02 -4.50632311e-02 -9.72411111e-02  9.45762638e-03
 -2.62959097e-02 -1.61208343e-02  3.52662988e-02  5.22052124e-02
 -4.16072011e-02 -1.09063894e-01 -2.76175123e-02 -1.82499178e-02
  1.89065263e-02 -2.74347141e-02  2.08957549e-02  3.61899026e-02
 -8.54327157e-03  4.21381146e-02 -2.20107753e-02  1.05593409e-02
 -1.02118207e-02 -4.45203111e-02 -9.09243152e-03  1.11676799e-02
 -1.51239103e-03  2.11241469e-02  1.68589447e-02  2.67801080e-02
  9.42392740e-03 -1.82698723e-02  8.39706585e-02  4.58729407e-03
  6.48139641e-02 -1.84567701e-02  8.70616268e-03 -2.43653748e-02
 -2.95284651e-02  5.53023852e-02  3.77450511e-02 -1.72798391e-02
  3.01802866e-02  7.03296158e-03  3.18123624e-02  4.00716253e-03
  3.63538675e-02  1.39773199e-02 -1.43893063e-03  4.91259526e-03
  2.42633373e-02 -2.27482039e-02  1.38052348e-02 -5.21439835e-02
 -5.34484349e-02  1.41305411e-02 -8.1486